## Preprocessing

###  Label Encoding : Converting Strings into Numbers

In [127]:
df$cut<-as.numeric(df$cut) 
df$color<-as.numeric(df$color)
df$clarity<-as.numeric(df$clarity)

### Scaling

In [129]:
df.scl<-data.frame(scale(df[,-7]))

In [131]:
df.scl$price<-df$price #타겟 포함

In [132]:
str(df.scl)

'data.frame':	53940 obs. of  10 variables:
 $ carat  : num  -1.2 -1.24 -1.2 -1.07 -1.03 ...
 $ cut    : num  0.9815 0.0859 -1.7053 0.0859 -1.7053 ...
 $ color  : num  -0.937 -0.937 -0.937 1.414 2.002 ...
 $ clarity: num  -1.245 -0.638 0.576 -0.031 -1.245 ...
 $ totdep : num  -0.174 -1.361 -3.385 0.454 1.082 ...
 $ table  : num  -1.1 1.586 3.376 0.243 0.243 ...
 $ length : num  -1.59 -1.64 -1.5 -1.36 -1.24 ...
 $ width  : num  -1.54 -1.66 -1.46 -1.32 -1.21 ...
 $ depth  : num  -1.57 -1.74 -1.74 -1.29 -1.12 ...
 $ price  : int  326 326 327 334 335 336 336 337 337 338 ...


### PCA
- At the previous EDA, `length(8)`, `width(9)`, `depth(10)` All three vars were highly correlated.

In [ ]:
library(psych)

In [137]:
pca<-principal(df.scl[,7:9], nfactors=3,rotate="varimax"); pca

Principal Components Analysis
Call: principal(r = df.scl[, 7:9], nfactors = 3, rotate = "varimax")
Standardized loadings (pattern matrix) based upon correlation matrix
        RC2  RC3  RC1 h2      u2 com
length 0.60 0.61 0.52  1 1.1e-16 3.0
width  0.55 0.76 0.36  1 3.3e-16 2.3
depth  0.76 0.53 0.36  1 2.2e-16 2.3

                       RC2  RC3  RC1
SS loadings           1.24 1.22 0.54
Proportion Var        0.41 0.41 0.18
Cumulative Var        0.41 0.82 1.00
Proportion Explained  0.41 0.41 0.18
Cumulative Proportion 0.41 0.82 1.00

Mean item complexity =  2.5
Test of the hypothesis that 3 components are sufficient.

The root mean square of the residuals (RMSR) is  0 
 with the empirical chi square  0  with prob <  NA 

Fit based upon off diagonal values = 1

- Use only RC2 + RC3 (Becuase Cumulative Var is 0.82 )

In [139]:
pca.scr<-data.frame(pca$scores[,1:2]) ; head(pca.scr)

RC2,RC3
-1.0593010,-0.8860740
-1.4009072,-1.0073271
-1.8408313,-0.5281591
-0.7074556,-0.8317483
-0.4535444,-0.8759844
-0.7932781,-1.0244476


In [145]:
#  Remove the col before.
df.scl<-df.scl[,-c(7:10)]   

In [148]:
# Bind the col about pca scores.
df.pca<-cbind(df.scl,pca.scr)

In [151]:
# Add a target
df.pca$price<- df$price

In [152]:
head(df.pca) 

carat,cut,color,clarity,totdep,table,RC2,RC3,price
-1.198157,0.98146422,-0.9371541,-1.24520353,-0.1740899,-1.0996618,-1.0593010,-0.8860740,326
-1.240350,0.08588829,-0.9371541,-0.63808915,-1.3607259,1.5855140,-1.4009072,-1.0073271,326
-1.198157,-1.70526357,-0.9371541,0.57613962,-3.3849872,3.3756312,-1.8408313,-0.5281591,327
-1.071577,0.08588829,1.4142590,-0.03097476,0.4541292,0.2429261,-0.7074556,-0.8317483,334
-1.029384,-1.70526357,2.0021123,-1.24520353,1.0823482,0.2429261,-0.4535444,-0.8759844,335
-1.177060,-0.80968764,2.0021123,1.18325400,0.7333376,-0.2046032,-0.7932781,-1.0244476,336


### Splitting the Dataset

In [153]:
set.seed(2022)

In [156]:
idx<-sample(2, nrow(df.pca), replace=TRUE, prob=c(0.7,0.3))

In [157]:
train<-df.pca[idx==1,]
test<-df.pca[idx==2,]

In [158]:
str(train)

'data.frame':	37592 obs. of  9 variables:
 $ carat  : num  -1.24 -1.2 -1.07 -1.03 -1.18 ...
 $ cut    : num  0.0859 -1.7053 0.0859 -1.7053 -0.8097 ...
 $ color  : num  -0.937 -0.937 1.414 2.002 1.414 ...
 $ clarity: num  -0.638 0.576 -0.031 -1.245 1.79 ...
 $ totdep : num  -1.361 -3.385 0.454 1.082 0.384 ...
 $ table  : num  1.586 3.376 0.243 0.243 -0.205 ...
 $ RC2    : num  -1.401 -1.841 -0.707 -0.454 -0.873 ...
 $ RC3    : num  -1.007 -0.528 -0.832 -0.876 -0.959 ...
 $ price  : int  326 327 334 335 336 337 337 340 342 345 ...


In [159]:
str(test)

'data.frame':	16348 obs. of  9 variables:
 $ carat  : num  -1.2 -1.18 -1.2 -1.05 -1.03 ...
 $ cut    : num  0.981 -0.81 -0.81 -1.705 0.981 ...
 $ color  : num  -0.937 2.002 0.826 2.002 2.002 ...
 $ clarity: num  -1.245 1.183 0.576 -0.638 -1.245 ...
 $ totdep : num  -0.174 0.733 -1.64 1.571 0.315 ...
 $ table  : num  -1.1 -0.205 1.586 -1.1 -1.547 ...
 $ RC2    : num  -1.059 -0.793 -1.381 -0.357 -0.673 ...
 $ RC3    : num  -0.886 -1.024 -0.669 -0.931 -0.756 ...
 $ price  : int  326 336 338 339 344 348 351 352 354 403 ...
